# PRIMEIRA AVALIAÇÃO DE ECONOMETRIA

## Passos Iniciais

In [1]:
# Análise de dados básica
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Econometria
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

# Estatística
from scipy import stats

In [2]:
# O pandas foi configurado para mostrar todas as colunas e as 100 primeiras linhas

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 10)     

In [3]:
# O caminho do arquivo (base de dados) foi guardado em uma variável

caminho_arquivo = "/home/ph-padrim/Área de trabalho/Trabalho de Econometria/rais_firma_2021_aula.csv"

# O arquivo CSV foi transformado em um dataframe

df = pd.read_csv(caminho_arquivo, encoding='UTF-8')  

# Criar a coluna com o quadrado da idade_med
df['idade_med_quadrado'] = df['idade_med'] ** 2

# Criar interação entre sexo e raça/cor (se forem numéricas)
# Se forem categóricas, primeiro crie dummies
df['sexo_raca_interacao'] = df['sexo_med'] * df['raca_cor_med']

# Versão alternativa com get_dummies
dummies = pd.get_dummies(df['nivel_tec'], prefix='nivel_tec')
dummies = dummies[[f'nivel_tec_{i}' for i in range(1, 6)]]  # Remove o nível 0
df = pd.concat([df, dummies], axis=1)

# Convertendo as dummies em inteiros

for nivel in range(1, 6):
    df[f'nivel_tec_{nivel}'] = (df['nivel_tec'] == nivel).astype(int)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ph-padrim/Área de trabalho/Trabalho de Econometria/rais_firma_2021_aula.csv'

In [ ]:


# Mostra as 10 primeiras linhas do dataframe

display(df) 


## Análise 

### Descrição das variáveis

In [ ]:
# Mostra uma descrição da variável dependente utilizada nas regressões ('remun_med_real_med')

print('===========================================================')
print('Descrição da variável Y (remuneração média real da firma)')
print(' ' \
'')
print(df['remun_med_real_med'].describe())
print('==========================================================')

# Platando o gráfico de densidade kernel da variável Y (Remuneração média real da firma)

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='remun_med_real_med',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel da Remuneração Média Real da Firma', fontsize=14, pad=20)
plt.xlabel('Remuneração Média Real (R$)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['remun_med_real_med'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: R${media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()


In [ ]:

# Mostra uma descrição da variável independente utilizada nas regressões ('idade_med')

print('================================================')
print('Descrição da variável X1 (idade média na firma)')
print(' ' \
'')
print(df['idade_med'].describe())
print('===============================================')

# Platando o gráfico de densidade kernel da variável X1 (Idade média)

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='idade_med',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel da Idade Média', fontsize=14, pad=20)
plt.xlabel('Idade Média', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['idade_med'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()




In [ ]:
# Mostra uma descrição da variável dependente utilizada nas regressões ('superior_med')

print('========================================================================')
print('Descrição da variável X2 (proporção de ensino superior ou mais na firma)')
print(' ' \
'')
print(df['superior_med'].describe())
print('=======================================================================')

# Platando o gráfico de densidade kernel da variável Y (Proporção de ensino superior ou mais na firma)

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='superior_med',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel da Proporção de Ensino Superior na Firma ', fontsize=14, pad=20)
plt.xlabel('Proporção de Ensino Superior ou Mais na Firma', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['superior_med'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()

In [ ]:
# Mostra uma descrição da variável dependente utilizada nas regressões ('sexo_med')

print('========================================================================')
print('Descrição da variável X3 (proporção de mulheres na firma)')
print(' ' \
'')
print(df['sexo_med'].describe())
print('=======================================================================')

# Platando o gráfico de densidade kernel da variável Y (Proporção de ensino superior ou mais na firma)

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='sexo_med',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel da Proporção de Mulheres na Firma ', fontsize=14, pad=20)
plt.xlabel('Proporção de Mulheres na Firma', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['sexo_med'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()

In [ ]:
# Mostra uma descrição da variável dependente utilizada nas regressões ('raca_cor_med')

print('========================================================================')
print('Descrição da variável X4 (proporção de negros na firma)')
print(' ' \
'')
print(df['raca_cor_med'].describe())
print('=======================================================================')

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='raca_cor_med',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel da Proporção de Negros na Firma ', fontsize=14, pad=20)
plt.xlabel('Proporção de Negros na Firma', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['raca_cor_med'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()

In [ ]:
# Mostra uma descrição da variável dependente utilizada nas regressões ('nivel_tec')

print('========================================================================')
print('Descrição da variável X5 (nivel técnico na firma)')
print(' ' \
'')
print(df['nivel_tec'].describe())
print('=======================================================================')

# Configurar o estilo do gráfico

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar o gráfico de densidade Kernel (KDE)

sns.kdeplot(
    data=df,
    x='nivel_tec',
    color='royalblue',
    fill=True,
    alpha=0.3,  # Transparência do preenchimento
    linewidth=1.5  # Espessura da linha
)

# Adicionar título e labels
plt.title('Densidade Kernel do Nível Técnico na Firma ', fontsize=14, pad=20)
plt.xlabel('Nível Técnico na Firma', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Adicionar linha da média
media = df['nivel_tec'].mean()
plt.axvline(media, color='red', linestyle='--', label=f'Média: {media:.2f}')
plt.legend()

# Mostrar o gráfico
plt.show()

### Analisando modelos

#### remuneção média real da firma = const + proporção de ensino superior ou mais

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'superior_med']].dropna()
X = dados['superior_med']  # Variável independente
y = dados['remun_med_real_med']  # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)  # X agora tem duas colunas: [const, Superior_med]

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Gráfico de dispersão com linha de regressão
plt.figure(figsize=(10, 6))
plt.scatter(X['superior_med'], y, alpha=0.5, label='Dados observados')
plt.plot(X['superior_med'], modelo.predict(), color='red', label='Linha de regressão')
plt.title('Dispersão e Regressão')
plt.xlabel('Proporção com Ensino Superior ou Mais')
plt.ylabel('Remuneração Média Real da Firma')
plt.legend()
plt.grid(True)
plt.show()

# Dados do modelo
residuos = modelo.resid

# Configuração do gráfico
plt.figure(figsize=(10, 6))
ax = sns.histplot(residuos, kde=True, stat="density", linewidth=0)

# Adicionar curva normal
mu, std = stats.norm.fit(residuos)
xmin, xmax = ax.get_xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

# Ajustar limites do eixo Y
plt.ylim(0, 0.0010)  # Fixa o eixo Y entre 0 e 0.0010

# Configurações adicionais
plt.title('Distribuição dos Resíduos', pad=20)
plt.xlabel('Resíduos')
plt.ylabel('Densidade')
plt.grid(True, alpha=0.3)
plt.tight_layout()

plt.show()

#### remuneção média real da firma = const + idade média

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med']].dropna()
X = dados['idade_med']  # Variável independente
y = dados['remun_med_real_med']  # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)  # X agora tem duas colunas: [const, Superior_med]

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Gráfico de dispersão com linha de regressão
plt.figure(figsize=(10, 6))
plt.scatter(X['idade_med'], y, alpha=0.5, label='Dados observados')
plt.plot(X['idade_med'], modelo.predict(), color='red', label='Linha de regressão')
plt.title('Regressão Linear: Remuneração vs. Idade Média')
plt.xlabel('Idade Média')
plt.ylabel('Remuneração Média Real (remun_med_real_med)')
plt.legend()
plt.grid(True)
plt.show()

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()

#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med']].dropna()
X = dados[['idade_med', 'superior_med']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit()

# Resultados da regressão
print(modelo.summary())

# Carregar os dados e ajustar o modelo
dados = df[['remun_med_real_med', 'idade_med', 'superior_med']].dropna()
X = dados[['idade_med', 'superior_med']]
y = dados['remun_med_real_med']

# Adicionar constante e ajustar modelo
X = sm.add_constant(X)
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Preparar dados para a superfície
idade_range = np.linspace(dados['idade_med'].min(), dados['idade_med'].max(), 20)
superior_range = np.linspace(dados['superior_med'].min(), dados['superior_med'].max(), 20)
idade_grid, superior_grid = np.meshgrid(idade_range, superior_range)

# Criar matriz X para predição
X_pred = pd.DataFrame({
    'const': 1,
    'idade_med': idade_grid.ravel(),
    'superior_med': superior_grid.ravel()
})

# Prever valores
predicted = modelo.predict(X_pred).values.reshape(idade_grid.shape)

# Plotar APENAS a superfície de regressão
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Superfície da regressão com gradiente de cor
surface = ax.plot_surface(
    idade_grid,
    superior_grid,
    predicted,
    cmap=cm.coolwarm,  # Gradiente de cores
    alpha=0.8,
    linewidth=0,
    antialiased=True
)

# Barra de cores
fig.colorbar(surface, shrink=0.5, aspect=5, label='Remuneração Média Real')

# Configurações do gráfico
ax.set_xlabel('Idade Média', fontsize=12)
ax.set_ylabel('Proporção com Superior Completo', fontsize=12)
ax.set_zlabel('Remuneração Média Real', fontsize=12)
ax.set_title('Superfície de Regressão 3D: Remuneração vs Idade e Educação', fontsize=14, pad=20)

# Ajustar ângulo de visualização
ax.view_init(elev=25, azim=45)  # Elevação e rotação
plt.tight_layout()
plt.show()

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()


#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais + (idade média)^2

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado']].dropna()
X = dados[['idade_med', 'superior_med', 'idade_med_quadrado']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()

#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais + (idade média)^2 + proporção do sexo feminino

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med']].dropna()
X = dados[['idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()

#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais + (idade média)^2 + proporção do sexo feminino + proporção de raça parda, preta e indígena

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med']].dropna()
X = dados[['idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()


#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais + (idade média)^2 + proporção do sexo feminino + proporção de raça parda, preta e indígena + (proporção do sexo feminino * proporção de raça parda, preta e indígena)

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao']].dropna()
X = dados[['idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Calcular VIF para cada variável
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns  # Inclui todas as variáveis independentes
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Exibir resultados
print("Fator de Inflação de Variância (VIF):")
print(vif_data.sort_values("VIF", ascending=False))

# Selecionar as variáveis
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao']].dropna()

# 1. Calcular a matriz de correlação
corr = dados.corr()

# 2. Criar máscara para mostrar apenas o triângulo superior
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 8))
sns.heatmap(corr, 
            mask=mask,  # Aplicar máscara
            annot=True,
            fmt=".2f",
            cmap='coolwarm',
            center=0,
            vmin=-1, vmax=1,
            linewidths=0.5,
            cbar_kws={'label': 'Coeficiente de Correlação'})

# Ajustar título e labels
plt.title('Matriz de Correlação (Triangular Superior)', pad=20, fontsize=15)
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Remover valores duplicados na diagonal
plt.gca().xaxis.tick_top()  # Mover labels para cima

plt.tight_layout()
plt.savefig('corr_triangular.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# 3. Opcional: Mostrar pares com maior correlação (absoluta)
print("\nTop 5 correlações (em valor absoluto):")
corr_unstack = corr.unstack().sort_values(key=lambda x: abs(x), ascending=False)
corr_unstack = corr_unstack[corr_unstack < 1]  # Remove autocorrelações
print(corr_unstack.head(5))

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()

#### remuneção média real da firma = const + idade média + proporção de ensino superior ou mais + (idade média)^2 + proporção do sexo feminino + proporção de negros + (proporção do sexo feminino * proporção de raça parda, preta e indígena) + nivel_tec_1 + nivel_tec_2 + nivel_tec_3 + nivel_tec_4 + nivel_tec_5   

In [ ]:
# Selecionar variáveis e remover missing values
dados = df[['remun_med_real_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5']].dropna()
X = dados[['idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5']]  # Variáveis independentes
y = dados['remun_med_real_med']           # Variável dependente

# Adicionar constante (intercepto) ao modelo
X = sm.add_constant(X)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit(cov_type='HC3')

# Resultados da regressão
print(modelo.summary())

# Calcular VIF para cada variável
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns  # Inclui todas as variáveis independentes
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Exibir resultados
print("Fator de Inflação de Variância (VIF):")
print(vif_data.sort_values("VIF", ascending=False))

# 1. Calcular a matriz de correlação
corr = dados.corr()

# 2. Criar máscara triangular superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# 3. Plotar o heatmap triangular
plt.figure(figsize=(12, 8))
sns.heatmap(corr, 
            mask=mask,  # Aplicar a máscara
            annot=True,
            fmt=".2f",
            cmap='coolwarm',
            center=0,
            vmin=-1, vmax=1,
            linewidths=0.5,
            cbar_kws={'label': 'Coeficiente de Correlação'})

# Ajustes estéticos
plt.title('Matriz de Correlação (Triangular Superior)', pad=20, fontsize=15)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('corr_triangular_completa.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()

# Dados do seu modelo
residuos = modelo.resid
valores_ajustados = modelo.predict()

# Configuração do layout dos gráficos
plt.figure(figsize=(15, 10))
plt.suptitle('Análise de Resíduos', y=1.02, fontsize=16)

# Gráfico 1: Histograma + Distribuição Normal
plt.subplot(2, 2, 2)
sns.histplot(residuos, kde=True, stat="density", linewidth=0)
mu, std = stats.norm.fit(residuos)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title('Distribuição dos Resíduos')
plt.xlabel('Resíduos')
plt.grid(True)
plt.tight_layout()

plt.show()

